# Uniaxial Channel Waveguide

This example shows how to find the modes of a birefringent (uniaxial) channel waveguide that is surrounded by an isotropic cladding layer. The extraordinary axis of the core region is chosen to point in the x direction, such that the permittivity tensor is diagonal, but not proportional to the identity matrix. This example also demonstrates how to use the mesh-building methods to describe waveguides that contain anisotropic materials, including structures that include isotropic and anisotropic regions.

In this example, we simulate the entire waveguide cross-section, without imposing symmetric boundaries in either x or y. Technically, because the geometry is symmetric, we could have reduced the computational domain by enforcing symmetry conditions. However, we keep the full domain for two reasons: 

- To demonstrate that the solver can compute multiple modes simultaneously, and 
- To allow easy comparision with another example (`uniaxial_channel_rotated.ipynb`), that considers the case where the c-axis of the material is rotated in the xy plane, making epsxy and epsyx non-zero. (In that scenario, symmetry conditions in x or y cannot be imposed.)

In [ ]:
# Enable automatic reloading of modules (IPython only)
%reload_ext autoreload
%autoreload 2 

# load required modules
import modesolver as mode
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
# Refractive indices:
n1 = 1.55           # cladding index
n2x = 2.156         # extraordinary index (core)
n2y = 2.232         # ordinary index (core)
n2z = 2.232         # ordinary index (core)

# channel waveguide dimensions
Rx = 0.30           # half-width in x
Ry = 0.20           # half-height in y
side = 0.2          # cladding thickness (to be stretched by 4X later)

dx = 0.0025         # grid size (x)
dy = dx             # grid size (y)

# Build waveguide mesh (epsxx, epsyy, epszz)
x,y,xc,yc,nx,ny,epsxx       = mode.waveguidemeshfull([n1,n2x,n1], [side,2*Ry,side], 2*Ry, Rx, side, dx, dy)
x,y,xc,yc,nx,ny,epsyy       = mode.waveguidemeshfull([n1,n2y,n1], [side,2*Ry,side], 2*Ry, Rx, side, dx, dy)
x,y,xc,yc,nx,ny,epszz,edges = mode.waveguidemeshfull([n1,n2z,n1], [side,2*Ry,side], 2*Ry, Rx, side, dx, dy, return_edges=True)

In [ ]:
# Stretch cladding mesh on all four boundaries by 4X (from 0.2 µm to 0.8 µm):
x,y,xc,yc,dx,dy = mode.stretchmesh(x,y,[80,80,80,80],[4,4,4,4])

In [ ]:
wavelength = 1.0   # vacuum wavelength
nmodes = 2         # here we calculate two modes

neff, _, _, _, Hx, Hy, Hz = mode.wgmodes(wavelength, n2y, nmodes, dx, dy, epsxx=epsxx, epsyy=epsyy, epszz=epszz, boundary='0000')
print("neff =", [f"{v:.6f}" for v in neff])

In [ ]:
# Plot Modes
levels = np.arange(-45,0,3)
fig, axes = plt.subplots(nmodes,2, figsize=(8,8))

for m in range(nmodes):
    axHx = axes[m,0]
    axHy = axes[m,1]

    axHx.pcolormesh(x,y,abs(Hx[:,:,m]),cmap='jet',vmax=1)
    axHx.contour(x,y,20*np.log10(abs(Hx[:,:,m])),levels,colors='white',negative_linestyles='solid', linewidths=1)
    axHx.set_aspect('equal')
    axHx.axis([min(x), max(x), min(y), max(y)])
    axHx.set_xlabel('x')
    axHx.set_ylabel('y')
    axHx.set_title(f'Hx (mode {m})')
    l = LineCollection(edges, colors='black', linewidths=1)
    axHx.add_collection(l)

    axHy.pcolormesh(x,y,abs(Hy[:,:,m]),cmap='jet',vmax=1)
    axHy.contour(x,y,20*np.log10(abs(Hy[:,:,m])),levels,colors='white',negative_linestyles='solid', linewidths=1)
    axHy.set_aspect('equal')
    axHy.axis([min(x), max(x), min(y), max(y)])
    axHy.set_xlabel('x')
    axHy.set_ylabel('y')
    axHy.set_title(f'Hy (mode {m})')
    l = LineCollection(edges, colors='black', linewidths=1)
    axHy.add_collection(l)

plt.show()